# Análisis Exploratorio de Datos (EDA)

Este notebook contiene el análisis exploratorio de datos del dataset de clima, incluyendo:
- Carga y exploración inicial de los datos
- Análisis de estructura y tipos de datos
- Estadísticas descriptivas
- Identificación de valores faltantes y outliers
- Análisis de correlaciones entre variables
- Preparación inicial de los datos

In [4]:
import pandas as pd

## 1. Carga de Datos

Cargaremos los datasets de entrenamiento y prueba para comenzar el análisis.

In [5]:
# Cargar datos de entrenamiento y prueba
train_data = pd.read_csv('../data/train.csv')
test_data = pd.read_csv('../data/test.csv')

print("Dimensiones del conjunto de entrenamiento:", train_data.shape)
print("Dimensiones del conjunto de prueba:", test_data.shape)
print("\nPrimeras 5 filas del conjunto de entrenamiento:")
print(train_data.head())

Dimensiones del conjunto de entrenamiento: (1462, 5)
Dimensiones del conjunto de prueba: (114, 5)

Primeras 5 filas del conjunto de entrenamiento:
         date   meantemp   humidity  wind_speed  meanpressure
0  2013-01-01  10.000000  84.500000    0.000000   1015.666667
1  2013-01-02   7.400000  92.000000    2.980000   1017.800000
2  2013-01-03   7.166667  87.000000    4.633333   1018.666667
3  2013-01-04   8.666667  71.333333    1.233333   1017.166667
4  2013-01-05   6.000000  86.833333    3.700000   1016.500000


## 2. Exploración Inicial de la Estructura

In [6]:
# Información general sobre el dataset
print("=== INFORMACIÓN DEL DATASET DE ENTRENAMIENTO ===")
print(train_data.info())
print("\n=== TIPOS DE DATOS ===")
print(train_data.dtypes)
print("\n=== VALORES FALTANTES ===")
print(train_data.isnull().sum())
print("\n=== VALORES DUPLICADOS ===")
print(f"Filas duplicadas: {train_data.duplicated().sum()}")

=== INFORMACIÓN DEL DATASET DE ENTRENAMIENTO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          1462 non-null   object 
 1   meantemp      1462 non-null   float64
 2   humidity      1462 non-null   float64
 3   wind_speed    1462 non-null   float64
 4   meanpressure  1462 non-null   float64
dtypes: float64(4), object(1)
memory usage: 57.2+ KB
None

=== TIPOS DE DATOS ===
date             object
meantemp        float64
humidity        float64
wind_speed      float64
meanpressure    float64
dtype: object

=== VALORES FALTANTES ===
date            0
meantemp        0
humidity        0
wind_speed      0
meanpressure    0
dtype: int64

=== VALORES DUPLICADOS ===
Filas duplicadas: 0


In [7]:
# Convertir la columna de fecha a datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Verificar el rango de fechas
print("=== RANGO DE FECHAS ===")
print(f"Entrenamiento: {train_data['date'].min()} a {train_data['date'].max()}")
print(f"Prueba: {test_data['date'].min()} a {test_data['date'].max()}")
print(f"Días totales en entrenamiento: {len(train_data)}")
print(f"Días totales en prueba: {len(test_data)}")

=== RANGO DE FECHAS ===
Entrenamiento: 2013-01-01 00:00:00 a 2017-01-01 00:00:00
Prueba: 2017-01-01 00:00:00 a 2017-04-24 00:00:00
Días totales en entrenamiento: 1462
Días totales en prueba: 114


## 3. Estadísticas Descriptivas

In [8]:
# Estadísticas descriptivas del conjunto de entrenamiento
print("=== ESTADÍSTICAS DESCRIPTIVAS - ENTRENAMIENTO ===")
print(train_data.describe())

print("\n=== ESTADÍSTICAS DESCRIPTIVAS - PRUEBA ===")
print(test_data.describe())

=== ESTADÍSTICAS DESCRIPTIVAS - ENTRENAMIENTO ===
                      date     meantemp     humidity   wind_speed  \
count                 1462  1462.000000  1462.000000  1462.000000   
mean   2015-01-01 12:00:00    25.495521    60.771702     6.802209   
min    2013-01-01 00:00:00     6.000000    13.428571     0.000000   
25%    2014-01-01 06:00:00    18.857143    50.375000     3.475000   
50%    2015-01-01 12:00:00    27.714286    62.625000     6.221667   
75%    2016-01-01 18:00:00    31.305804    72.218750     9.238235   
max    2017-01-01 00:00:00    38.714286   100.000000    42.220000   
std                    NaN     7.348103    16.769652     4.561602   

       meanpressure  
count   1462.000000  
mean    1011.104548  
min       -3.041667  
25%     1001.580357  
50%     1008.563492  
75%     1014.944901  
max     7679.333333  
std      180.231668  

=== ESTADÍSTICAS DESCRIPTIVAS - PRUEBA ===
                      date    meantemp    humidity  wind_speed  meanpressure
count    

## 4. Análisis de Outliers

In [9]:
# Función para detectar outliers usando el método IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Analizar outliers en cada variable numérica
numeric_columns = ['meantemp', 'humidity', 'wind_speed', 'meanpressure']

print("=== ANÁLISIS DE OUTLIERS ===")
for col in numeric_columns:
    outliers, lower, upper = detect_outliers_iqr(train_data, col)
    print(f"\n{col.upper()}:")
    print(f"  Rango normal: [{lower:.2f}, {upper:.2f}]")
    print(f"  Outliers detectados: {len(outliers)}")
    if len(outliers) > 0:
        print(f"  Valores outliers: {outliers[col].values[:5]}...")  # Mostrar solo los primeros 5

=== ANÁLISIS DE OUTLIERS ===

MEANTEMP:
  Rango normal: [0.18, 49.98]
  Outliers detectados: 0

HUMIDITY:
  Rango normal: [17.61, 104.98]
  Outliers detectados: 2
  Valores outliers: [15.85714286 13.42857143]...

WIND_SPEED:
  Rango normal: [-5.17, 17.88]
  Outliers detectados: 30
  Valores outliers: [24.06666667 18.525      42.22       19.9125     34.4875    ]...

MEANPRESSURE:
  Rango normal: [981.53, 1034.99]
  Outliers detectados: 9
  Valores outliers: [7679.33333333  938.06666667  946.3125      310.4375      633.9       ]...


## 5. Análisis de Correlaciones

In [10]:
# Matriz de correlación
correlation_matrix = train_data[numeric_columns].corr()

print("=== MATRIZ DE CORRELACIÓN ===")
print(correlation_matrix.round(3))

# Identificar correlaciones fuertes
print("\n=== CORRELACIONES FUERTES (|r| > 0.7) ===")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            var1 = correlation_matrix.columns[i]
            var2 = correlation_matrix.columns[j]
            print(f"{var1} - {var2}: {corr_val:.3f}")

=== MATRIZ DE CORRELACIÓN ===
              meantemp  humidity  wind_speed  meanpressure
meantemp         1.000    -0.572       0.306        -0.039
humidity        -0.572     1.000      -0.374         0.002
wind_speed       0.306    -0.374       1.000        -0.021
meanpressure    -0.039     0.002      -0.021         1.000

=== CORRELACIONES FUERTES (|r| > 0.7) ===


## 6. Análisis Temporal

In [11]:
# Crear características temporales
train_data['year'] = train_data['date'].dt.year
train_data['month'] = train_data['date'].dt.month
train_data['day_of_year'] = train_data['date'].dt.dayofyear
train_data['weekday'] = train_data['date'].dt.dayofweek

# Análisis por año
print("=== ANÁLISIS POR AÑO ===")
yearly_stats = train_data.groupby('year')[numeric_columns].agg(['mean', 'std'])
print(yearly_stats.round(2))

# Análisis por mes
print("\n=== ANÁLISIS POR MES (TEMPERATURA PROMEDIO) ===")
monthly_temp = train_data.groupby('month')['meantemp'].agg(['mean', 'std'])
print(monthly_temp.round(2))

=== ANÁLISIS POR AÑO ===
     meantemp       humidity        wind_speed       meanpressure        
         mean   std     mean    std       mean   std         mean     std
year                                                                     
2013    24.79  7.41    63.05  18.19       6.83  4.83      1007.64    7.76
2014    25.01  7.60    59.77  16.22       6.76  4.66      1008.35    7.64
2015    25.11  7.24    61.43  15.69       6.48  4.70      1008.83    7.43
2016    27.10  6.89    58.74  16.51       7.16  3.99      1019.56  360.21
2017    10.00   NaN   100.00    NaN       0.00   NaN      1016.00     NaN

=== ANÁLISIS POR MES (TEMPERATURA PROMEDIO) ===
        mean   std
month             
1      13.31  2.47
2      17.62  2.92
3      22.91  3.26
4      29.38  3.01
5      33.32  2.79
6      33.73  2.70
7      31.00  2.19
8      30.60  1.78
9      30.43  1.51
10     27.11  2.64
11     20.66  2.43
12     15.67  3.13


## 7. Resumen de Hallazgos

In [12]:
print("=== RESUMEN DEL ANÁLISIS EXPLORATORIO ===")
print("1. ESTRUCTURA DE DATOS:")
print(f"   - {len(train_data)} registros de entrenamiento, {len(test_data)} de prueba")
print(f"   - {len(numeric_columns)} variables climáticas principales")
print(f"   - Rango temporal: {train_data['date'].min().strftime('%Y-%m-%d')} a {test_data['date'].max().strftime('%Y-%m-%d')}")

print("\n2. CALIDAD DE DATOS:")
print(f"   - Valores faltantes: {train_data[numeric_columns].isnull().sum().sum()} en total")
print(f"   - Valores duplicados: {train_data.duplicated().sum()}")

print("\n3. CARACTERÍSTICAS PRINCIPALES:")
print(f"   - Temperatura media: {train_data['meantemp'].mean():.1f}°C (±{train_data['meantemp'].std():.1f})")
print(f"   - Humedad promedio: {train_data['humidity'].mean():.1f}% (±{train_data['humidity'].std():.1f})")
print(f"   - Velocidad viento: {train_data['wind_speed'].mean():.1f} (±{train_data['wind_speed'].std():.1f})")
print(f"   - Presión media: {train_data['meanpressure'].mean():.1f} (±{train_data['meanpressure'].std():.1f})")

print("\n4. PRÓXIMOS PASOS:")
print("   - Visualización detallada de tendencias y patrones")
print("   - Modelado predictivo para series temporales")
print("   - Análisis de estacionalidad y ciclos")

=== RESUMEN DEL ANÁLISIS EXPLORATORIO ===
1. ESTRUCTURA DE DATOS:
   - 1462 registros de entrenamiento, 114 de prueba
   - 4 variables climáticas principales
   - Rango temporal: 2013-01-01 a 2017-04-24

2. CALIDAD DE DATOS:
   - Valores faltantes: 0 en total
   - Valores duplicados: 0

3. CARACTERÍSTICAS PRINCIPALES:
   - Temperatura media: 25.5°C (±7.3)
   - Humedad promedio: 60.8% (±16.8)
   - Velocidad viento: 6.8 (±4.6)
   - Presión media: 1011.1 (±180.2)

4. PRÓXIMOS PASOS:
   - Visualización detallada de tendencias y patrones
   - Modelado predictivo para series temporales
   - Análisis de estacionalidad y ciclos
